In [1]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import OrderedDict
import numpy as np
import re
import h5py
from functools import partial
from itertools import product

from calibration import pulse_filter, parse_ids, DataProcessing
from karabo_data import by_index

from karabo_data.geometry2 import AGIPD_1MGeometry
import time


In [2]:
module_numbers = [15, 14]

In [ ]:
dark_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0504"

def f(x):
    return x

dark_average = partial(
    DataProcessing,
    path=dark_folder,
    pulse_ids="1:250:2",
    train_index=by_index[:],
    rois=by_index[..., :, :],
    operation=partial(np.mean, axis=0))


futures = OrderedDict()

with ProcessPoolExecutor(max_workers=len(module_numbers)) as executor:
    for mod_no in module_numbers:
        futures[mod_no] = executor.submit(dark_average, mod_no)


In [ ]:
print(futures)

dark_data = { module:result.result() for module, result in futures.items()}

print(dark_data[15].shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


for idx, modno in enumerate(module_numbers):
    fig, ax = plt.subplots(1,2, figsize=(15,4), tight_layout=True)
    ax[0].imshow(dark_data[modno][0, 0, ...])
    ax[1].hist(dark_data[modno][:, 0, ...].ravel(), bins=100,  edgecolor='w')
    ax[0].set_aspect('auto')

In [ ]:
flat_field_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0491"

dark_subtraction = partial(
    DataProcessing,
    path=flat_field_folder,
    pulse_ids="1:250:2",
    train_index=by_index[250:280],
    rois=by_index[..., :, :],
    operation=partial(f),
    dark_run=dark_data)

futures_dark_sub = OrderedDict()

with ProcessPoolExecutor(max_workers=len(module_numbers)) as executor:
    for mod_no in module_numbers:
        futures_dark_sub[mod_no] = executor.submit(dark_subtraction, mod_no)

In [ ]:
print(futures_dark_sub)
dark_subtracted_data = { module:result.result() for module, result in futures_dark_sub.items()}

dark_subtracted_data[15].shape

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter, gaussian_filter1d


%matplotlib inline

for idx in range(95, 100):
    fig, ax = plt.subplots(1,2, figsize=(15,4), tight_layout=True)
    ax[0].imshow(dark_subtracted_data[15][0, idx, 0, ...] )
    counts, bins, _ = ax[1].hist(dark_subtracted_data[15][:, idx, 0 , ...].ravel(), bins=1000)
    ax[0].set_aspect('auto')
    bin_centers = (bins[1:] + bins[:-1])/2.0
    #filtered = savgol_filter(counts, 21, 2)
    filtered = gaussian_filter(counts, 1.5)
    peaks, _ = find_peaks(filtered, height=200)
    ax[1].plot(bin_centers[peaks], filtered[peaks], 'x')
    ax[1].plot(bin_centers, filtered)